# 自动微分
# 自动求导
:label:`sec_autograd`

正如我们在 :numref:`sec_calculus`中所说的那样，求导是几乎所有深度学习优化算法的关键步骤。
虽然求导的计算很简单，只需要一些基本的微积分。
但对于复杂的模型，手工进行更新是一件很痛苦的事情（而且经常容易出错）。

深度学习框架通过自动计算导数，即*自动微分*（automatic differentiation）来加快求导。
实际中，根据我们设计的模型，系统会构建一个*计算图*（computational graph），
来跟踪计算是哪些数据通过哪些操作组合起来产生输出。
自动微分使系统能够随后反向传播梯度。
这里，*反向传播*（backpropagate）意味着跟踪整个计算图，填充关于每个参数的偏导数。

## 一个简单的例子

作为一个演示例子，(**假设我们想对函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于列向量$\mathbf{x}$求导**)。
首先，我们创建变量`x`并为其分配一个初始值。


In [8]:
import torch # import torch. 导入torch

x = torch.arange(4.0) # 生成一个序列.  
x

tensor([0., 1., 2., 3.])

[**在我们计算$y$关于$\mathbf{x}$的梯度之前，我们需要一个地方来存储梯度。**]
重要的是，我们不会在每次对一个参数求导时都分配新的内存。
因为我们经常会成千上万次地更新相同的参数，每次都分配新的内存可能很快就会将内存耗尽。
注意，一个标量函数关于向量$\mathbf{x}$的梯度是向量，并且与$\mathbf{x}$具有相同的形状。


In [9]:
# 在我们计算 𝑦 关于 𝐱 的梯度之前，我们需要一个地方来存储梯度。
# 因此, 我们需要指定: requires_grad()来告诉torch, 这个变量我需要保存梯度. 然后我们就可以通过.grad查看梯度了。

x.requires_grad_(True)  # 等价于x=torch.arange(4.0,requires_grad=True)
x.grad  # 默认值是None 
print(x.grad)


None


(**现在让我们计算$y$。**)


In [10]:
y = 2 * torch.dot(x, x) # 就是2*x^2
# 求导后: 4*x. 
y
# 因为是隐式的求解梯度, 因此这里是有一个函数: grad_fn来求解梯度. 

tensor(28., grad_fn=<MulBackward0>)

`x`是一个长度为4的向量，计算`x`和`x`的点积，得到了我们赋值给`y`的标量输出。
接下来，我们[**通过调用反向传播函数来自动计算`y`关于`x`每个分量的梯度**]，并打印这些梯度。


In [11]:
y.backward() # 调用反向传播来查看y关于x的每一个分量的梯度. 
x.grad # 通过x.grad访问一下在x每个分量的导数.

tensor([ 0.,  4.,  8., 12.])

函数$y=2\mathbf{x}^{\top}\mathbf{x}$关于$\mathbf{x}$的梯度应为$4\mathbf{x}$。
让我们快速验证这个梯度是否计算正确。


In [12]:
x.grad == 4 * x

tensor([True, True, True, True])

[**现在让我们计算`x`的另一个函数。**]


In [21]:
# 在默认情况下，PyTorch会累积梯度，所以我们需要清除之前的值
x.grad.zero_() # 在pytorch中_表示原地操作, 表示对当前变量进行原地操作. 
# 对梯度清零.
print(x.grad) #
# 重新计算: y的函数: 
y = x.sum() # 标量.
y.backward() # backward calculate
x.grad # get gradient

tensor([0., 0., 0., 0.])


tensor([1., 1., 1., 1.])

## 非标量变量的反向传播

当`y`不是标量时，向量`y`关于向量`x`的导数的最自然解释是一个矩阵。
对于高阶和高维的`y`和`x`，求导的结果可以是一个高阶张量。

然而，虽然这些更奇特的对象确实出现在高级机器学习中（包括[**深度学习中**]），
但当我们调用向量的反向计算时，我们通常会试图计算一批训练样本中每个组成部分的损失函数的导数。
这里(**，我们的目的不是计算微分矩阵，而是单独计算批量中每个样本的偏导数之和。**)


In [14]:
# 对非标量调用backward需要传入一个gradient参数，该参数指定微分函数关于self的梯度。
# 在我们的例子中，我们只想求偏导数的和，所以传递一个1的梯度是合适的
x.grad.zero_() # 清空. 
y = x * x # 这是一个向量. 默认是元素运算. 这不是一个标量.
# 那么理论上, 他的梯度是一个矩阵. 但是在深度学习中我们很少做这个事情.
# 等价于y.backward(torch.ones(len(x)))
y.sum().backward() # 我们在绝大多数情况都是对一个标量进行求导, 我们不会对一个vector或者一个matrix求导. 
# 因此我们会先使用sum()将任何维度转换成一个标量. 然后在求导. 
y, x.grad
# 我们后面会讲在什么情况下进行求和.

(tensor([0., 1., 4., 9.], grad_fn=<MulBackward0>), tensor([0., 2., 4., 6.]))

## 分离计算

有时，我们希望[**将某些计算移动到记录的计算图之外**]。
例如，假设`y`是作为`x`的函数计算的，而`z`则是作为`y`和`x`的函数计算的。
想象一下，我们想计算`z`关于`x`的梯度，但由于某种原因，我们希望将`y`视为一个常数，
并且只考虑到`x`在`y`被计算后发挥的作用。

在这里，我们可以分离`y`来返回一个新变量`u`，该变量与`y`具有相同的值，
但丢弃计算图中如何计算`y`的任何信息。
换句话说，梯度不会向后流经`u`到`x`。
因此，下面的反向传播函数计算`z=u*x`关于`x`的偏导数，同时将`u`作为常数处理，
而不是`z=x*x*x`关于`x`的偏导数。


In [15]:
# 我可以将一些计算移动到计算图之外. 
x.grad.zero_() # empty gradient
y = x * x # definite y function
u = y.detach() # detach抽取出来. 就是y看作一个scalar. 是一个常数, 
# y.detach()此时就不是一个关于x的函数, 而仅仅是x*x的一个数
z = u * x # z就是一个常数*u
# 那么求导之后, 关于x的导数就是u

z.sum().backward()
x.grad == u

tensor([True, True, True, True])

关于.detach(), 就是cut branch. 剪枝操作, 我们不需要一些网络中的参数时候, 或者我们需要将一些参数固定住的时候, 那么detach是非常有用的. 
detach: 拆下, 使分离, 脱离, 摆脱. 

由于记录了`y`的计算结果，我们可以随后在`y`上调用反向传播，
得到`y=x*x`关于的`x`的导数，即`2*x`。


In [16]:
x.grad.zero_()
# 此时y还是x的函数, 因此对y进行backward后, 得到的还是关于x的导数. 
y.sum().backward()
x.grad == 2 * x

tensor([True, True, True, True])

## Python控制流的梯度计算

使用自动微分的一个好处是：
[**即使构建函数的计算图需要通过Python控制流（例如，条件、循环或任意函数调用），我们仍然可以计算得到的变量的梯度**]。
在下面的代码中，`while`循环的迭代次数和`if`语句的结果都取决于输入`a`的值。


In [17]:
def f(a):
    b = a * 2
    while b.norm() < 1000: # norm()就是范数. 
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

In [18]:
def f(a):
    b = a * 2
    while b.norm() < 1000: # norm()就是范数. 
        b = b * 2
    if b.sum() > 0:
        c = b
    else:
        c = 100 * b
    return c

让我们计算梯度。


In [19]:
a = torch.randn(size=(), requires_grad=True)
d = f(a)
d.backward()

我们现在可以分析上面定义的`f`函数。
请注意，它在其输入`a`中是分段线性的。
换言之，对于任何`a`，存在某个常量标量`k`，使得`f(a)=k*a`，其中`k`的值取决于输入`a`。
因此，我们可以用`d/a`验证梯度是否正确。


In [20]:
a.grad == d / a

tensor(True)

## 小结

* 深度学习框架可以自动计算导数：我们首先将梯度附加到想要对其计算偏导数的变量上。然后我们记录目标值的计算，执行它的反向传播函数，并访问得到的梯度。

## 练习

1. 为什么计算二阶导数比一阶导数的开销要更大？
1. 在运行反向传播函数之后，立即再次运行它，看看会发生什么。
1. 在控制流的例子中，我们计算`d`关于`a`的导数，如果我们将变量`a`更改为随机向量或矩阵，会发生什么？
1. 重新设计一个求控制流梯度的例子，运行并分析结果。
1. 使$f(x)=\sin(x)$，绘制$f(x)$和$\frac{df(x)}{dx}$的图像，其中后者不使用$f'(x)=\cos(x)$。


[Discussions](https://discuss.d2l.ai/t/1759)
